In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
accepted = pd.read_csv("data/accepted_2007_to_2018Q4.csv")
# pd.set_option('display.max_columns', None)

In [ ]:
# TODO: change method to len of words (not len of char)
accepted["desc_len"] = accepted.dropna(subset="desc")["desc"].apply(lambda x: len(str(x).split()))

In [ ]:
date_columns = ["issue_d", "hardship_start_date", "hardship_end_date", "payment_plan_start_date", "debt_settlement_flag_date", "settlement_date"]

In [ ]:
date_columns = ["issue_d", "hardship_start_date", "hardship_end_date", "payment_plan_start_date", "debt_settlement_flag_date", "settlement_date", "earliest_cr_line"]
for c in date_columns: 
    res = list(map(lambda x: str(x).split("-"), accepted[c].values.tolist()))
    not_nan_dates = [ date for date in res if 'nan' not in date ]
    args = [ idx for idx, [_, y] in enumerate(not_nan_dates) if int(y) <= 2014 ]
    print(f"{c}: {len(args)}")

In [ ]:
c = "issue_d"
res = list(map(lambda x: str(x).split("-"), accepted[c].values.tolist()))
not_nan_dates = [ date for date in res if 'nan' not in date ]
args = [ idx for idx, [_, y] in enumerate(not_nan_dates) if int(y) <= 2014 and int(y) >= 2007]
print(f"{c}: {len(args)}")

In [ ]:
data_2007_2014 = accepted.iloc[args]

In [ ]:
data_2007_2014_len_20 = data_2007_2014[data_2007_2014["desc_len"] > 20]

In [ ]:
from src.data.features import features

In [ ]:
data_ = data_2007_2014_len_20[features.keys()].dropna()

In [ ]:
data_

In [ ]:
data_["purpose"].unique()

In [ ]:
features.pop("total_il_high_credit_limit")
features.pop("revol_util")
features

In [ ]:
data_ = data_2007_2014_len_20[features.keys()].dropna()
data_

In [ ]:
data_["loan_status"].unique()

In [ ]:
data_["target"] = data_["loan_status"].apply(lambda x: "Fully Paid" in x)

In [ ]:
data_[data_["target"]]["desc_len"].describe(), data_[data_["target"]==False]["desc_len"].describe(), data_["desc_len"].describe()

In [ ]:
# date_columns = ["hardship_start_date", "hardship_end_date", "payment_plan_start_date", "debt_settlement_flag_date", "settlement_date", "earliest_cr_line"]
# for c in date_columns: 
#     res = list(map(lambda x: str(x).split("-"), accepted[c].values.tolist()))
#     not_nan_dates = [ date for date in res if 'nan' not in date ]
#     years = [ int(y) for _, y in not_nan_dates ]
#     print(f"{c}: {sum(years) / len(years)}")

In [ ]:
from src.data.preprocess import preprocess_data

preprocess_data()

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import stanza

stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline
doc = nlp("Barack Obama was born in Hawaii.")

In [ ]:
doc